# Video Translator

<a target="_blank" href="https://colab.research.google.com/github/imkasen/video-translator/blob/main/video_translator.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

使用 Faster Whisper 和 Google Gemini 进行字幕生成与翻译。

在 Google Colab 环境下：

1. 使用 Faster Whisper 对视频 / 音频进行语音识别，并生成字幕文件
2. 利用 Google Gemini 对源字幕进行翻译，并生成双语字幕文件

**你需要准备一个 Google Gemini API。**

In [ ]:
# @title ### 0. 地区查询
# @markdown 查询当前 Colab 所在地区是否可以使用 Gemini 免费版。

# @markdown [API 可用国家/地区查询](https://ai.google.dev/gemini-api/docs/available-regions)

!curl ipinfo.io

In [ ]:
# @title ### 1. 处理器类型
# @markdown **推荐在 Colab 中使用 T4 GPU 运行环境。**

!nvidia-smi

import torch

device: str = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n当前环境使用 '{device}'。")

In [ ]:
# @title ## 2. 安装依赖
%pip install --upgrade --quiet ffmpeg openai-whisper faster-whisper langchain langchain-google-genai

In [ ]:
# @title ## 3. 上传视频 / 音频文件

from google.colab import files

uploaded: dict = files.upload()

uploaded_filename: str = ""
if len(list(uploaded.keys())) == 1:
    uploaded_filename: str = list(uploaded.keys())[0]

## 4. 提取音频文件

In [ ]:
# @title ### 4.1 提取音频
# @markdown 使用 FFmpeg 从视频中提取音频文件。

import os
import mimetypes


if " " in uploaded_filename:
    new_filename = uploaded_filename.replace(" ", "_")
    os.rename(uploaded_filename, new_filename)
    print(f"文件已从 '{uploaded_filename}' 重命名为 '{new_filename}'")
    uploaded_filename = new_filename

prefix_filename, _ = os.path.splitext(uploaded_filename)
mime_type, _ = mimetypes.guess_type(uploaded_filename)
audio_filename: str = ""

if mime_type and mime_type.startswith("video/"):
    os.system(f'ffmpeg -hide_banner -v error -i "{uploaded_filename}" -vn -c:a copy "{prefix_filename}.aac" -y')
    audio_filename = f"{prefix_filename}.aac"
    if os.path.isfile(audio_filename):
        print(f'提取音频文件："{audio_filename}"')
    else:
        print(f'提取音频文件 "{audio_filename}" 失败！')
elif mime_type and mime_type.startswith("audio/"):
    audio_filename = uploaded_filename
    print(f'音频文件为："{audio_filename}"')
else:
    print("未能识别上传的文件格式！")

In [ ]:
# @title ### 4.2 下载音频文件
# @markdown 下载提取的音频文件到浏览器默认下载路径。

from google.colab import files

files.download(audio_filename)

## 5. 语音识别

调用 Faster Whisper 进行语音转文字识别。

In [ ]:
# @title ### 5.1 下载模型

from faster_whisper import WhisperModel

compute_type: str = ""
if device == "cuda":
    compute_type = "float16"
if device == "cpu":
    compute_type = "int8"
model = WhisperModel("large-v3", device=device, compute_type=compute_type)

In [ ]:
# @title ### 5.2 导出字幕

from typing import Any
from whisper.utils import get_writer

segments, _ = model.transcribe(
    audio_filename,
    beam_size=5,
    initial_prompt="请使用简体中文来显示中文汉字内容。",
    vad_filter=True,
    vad_parameters={"min_silence_duration_ms": 500},
)

segs_lst: list = []
text: str = ""
for segment in segments:
    text += segment.text
    segment_dict: dict[str, Any] = segment._asdict()
    segment_dict.pop("words")
    segs_lst.append(segment_dict)
result: dict[str, str | list] = {"text": text, "segments": segs_lst}


# @markdown 使用 openai-whisper 中的 `writer` 方法来保存字幕文件到当前运行环境下。
writer = get_writer("srt", ".")
writer(result, prefix_filename)

In [ ]:
# @title ### 5.3 下载字幕
# @markdown 下载字幕文件到浏览器默认下载路径。

from google.colab import files

files.download(f"{prefix_filename}.srt")

## 6. 翻译

---

导入 Google Gemini 密钥，以下命令**选择其中之一执行**即可。

In [ ]:
# @title ### 6.1 (1) 手动填写 Gemini API Key

gemini_api_key = ""  # @param {type: "string"}

In [ ]:
# @title ### 6.1 (2) 从 Colab Secret 中读取 Gemini API Key

from google.colab import userdata

# @markdown **请填入 Secret 名称：**
key_name = "API_KEY"  # @param {type: "string"}
gemini_api_key = userdata.get(key_name)

In [ ]:
# @title ### 6.2 请输入需要翻译为哪种语言：

language = "简体中文"  # @param {type: "string"}

---

基于 LangChain 调用 Google Gemini 对字幕文件进行翻译。

In [ ]:
# @title ### 6.3 提示词模板

template: str = r"""
你是一名精通任何语言的翻译专家，尤其擅长将任何外语翻译为{language}。请你帮我翻译字幕文件内容。

# 规则

- 翻译时要准确传达原文的事实和背景。
- 即使上意译也要保留原始段落格式，以及保留术语，例如 FLAC，JPEG 等。保留公司缩写，例如 Microsoft, Amazon, OpenAI 等。
- 人名不翻译
- 全角括号换成半角括号，并在左括号前面加半角空格，右括号后面加半角空格。
- 中文、外语、数字、字符混用时彼此之间添加空格
- 输入与输出格式保持一致，例如输入格式为 srt 格式，输出格式也必须为 srt 格式
- 在翻译专业术语时，第一次出现时要在括号里面写上英文原文，例如：“生成式 AI (Generative AI)”，之后就可以只写中文了。
- 以下是常见的 AI 相关术语词汇对应表（English -> 中文）：
  * Transformer -> Transformer
  * Token -> Token
  * LLM/Large Language Model -> 大语言模型
  * Zero-shot -> 零样本
  * Few-shot -> 少样本
  * AI Agent -> AI 智能体
  * AGI -> 通用人工智能
  * Prompt -> 提示词
  * LLMs -> LLMs

# 策略

分以下几步进行翻译工作：
1. 用户发送给你的字幕内容可能包含多种语言，如英文、西班牙语混合，并且字幕内容可能由机器识别生成，例如通过 OpenAI Whisper 框架识别产生，
   所以请先检查原文内容，并根据前后文本修正其中存在的问题，包括但不限于：
   - 语法错误
   - 内容缺失造成的语句不通顺
2. 根据内容直译，保持原有格式，不要遗漏任何信息
3. 根据直译的结果找出问题，包括但不限于：
   - 不符合中文表达习惯，指出不符合的地方
   - 语句不通顺，指出位置，不需要给出修改意见，意译时修复
   - 晦涩难懂，不易理解，可以尝试给出解释
4. 根据直译结果和指出的问题重新进行意译，保证内容的原意的基础上，使其更易于理解，更符合中文的表达习惯

# 示例

要求保持原有的字幕格式不变，生成多语种字幕，并把{language}作为主要字幕，把原语言作为次要字幕，然后返回翻译结果。

以下是把一个英文 srt 字幕片段翻译为中文英文双语字幕的示例，在后续翻译过程中你需要参考该示例的格式。

## 示例 - 原始字幕内容

```
1
00:00:00,000 --> 00:00:02,900
Hello, world!
```

## 示例 - 翻译结果

```
1
00:00:00,000 --> 00:00:02,900
你好，世界！
Hello, world!
```

# 你需要翻译的字幕内容：

```
{content}
```

# 输出格式要求

返回的输出不需要使用 Markdown 代码块格式，即不要使用 ```，直接返回内容即可。
"""

In [ ]:
# @title ### 6.4 使用 Google Gemini 模型

from langchain_google_genai import GoogleGenerativeAI

# @markdown 选择模型：
model_name = "gemini-1.5-pro-latest"  # @param ["gemini-1.5-pro-latest", "gemini-1.5-flash-latest"]
llm = GoogleGenerativeAI(model=model_name, google_api_key=gemini_api_key)

In [ ]:
# @title ### 6.5 翻译字幕
# @markdown 翻译并保存双语字幕到当前 Colab 运行环境下。

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

# 读取字幕文件
subtitle_content = ""
with open(f"{prefix_filename}.srt", "r", encoding="utf-8") as source_subtitle:
    subtitle_content = source_subtitle.read()
    print(f"读取源字幕文件：{prefix_filename}.srt")

response = chain.invoke({"language": language, "content": subtitle_content})

# 保存字幕文件
output_subtitle_name = f"Translated_{prefix_filename}.srt"
with open(output_subtitle_name, "w", encoding="utf-8") as output_subtitle:
    output_subtitle.write(response)
    print(f"保存翻译后的字幕文件：{output_subtitle_name}")

In [ ]:
# @title ### 6.6 下载字幕
# @markdown 下载翻译后的字幕文件到浏览器默认下载路径。

from google.colab import files

files.download(output_subtitle_name)

In [ ]:
# @title ### *6.7 查看翻译后的字幕内容（可选）

print(response)